In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

CSV_PATH = "orders (1).csv"   # change if needed

def clean_currency(series: pd.Series) -> pd.Series:
    """
    Converts values like '$5,478.67' -> 5478.67
    Works even if the column is already numeric.
    """
    if series.dtype == "O":
        series = (
            series.astype(str)
            .str.replace("$", "", regex=False)
            .str.replace(",", "", regex=False)
            .str.strip()
        )
    return pd.to_numeric(series, errors="coerce")

# -------------------
# Load + clean
# -------------------
df = pd.read_csv(CSV_PATH)

needed_cols = ["Product name", "Copies", "Charged", "Charged amount"]
missing = [c for c in needed_cols if c not in df.columns]
if missing:
    raise ValueError(f"Missing columns in CSV: {missing}")

df["Copies"] = pd.to_numeric(df["Copies"], errors="coerce").fillna(0)
df["Charged"] = df["Charged"].astype(str).str.strip().str.title()
df.loc[~df["Charged"].isin(["Yes", "No"]), "Charged"] = "Unknown"

df["Charged amount"] = clean_currency(df["Charged amount"])

# -------------------
# 1) Matrix/Table: Product totals
# -------------------
product_summary = (
    df.groupby("Product name", dropna=False)
      .agg(
          **{
              "Sum of Charged amount": ("Charged amount", "sum"),
              "Sum of Copies": ("Copies", "sum"),
          }
      )
      .sort_values("Sum of Charged amount", ascending=False)
)

# Add Total row like Power BI
total_row = pd.DataFrame(
    {
        "Sum of Charged amount": [product_summary["Sum of Charged amount"].sum()],
        "Sum of Copies": [product_summary["Sum of Copies"].sum()],
    },
    index=["Total"]
)
product_summary_with_total = pd.concat([product_summary, total_row])

print("\n=== POWER BI MATRIX (replica) ===")
print(product_summary_with_total.round(2))

# -------------------
# 2) Column chart: Count of Charged by Charged
# -------------------
count_charged = df["Charged"].value_counts().reindex(["No", "Yes", "Unknown"]).fillna(0)

plt.figure()
plt.bar(count_charged.index, count_charged.values)
plt.title("Count of Charged by Charged")
plt.xlabel("Charged")
plt.ylabel("Count of Charged")
plt.tight_layout()
plt.show()

# -------------------
# 3) Bar chart: Sum of Copies by Product name
# -------------------
copies_by_product = df.groupby("Product name")["Copies"].sum().sort_values()

plt.figure()
plt.barh(copies_by_product.index, copies_by_product.values)
plt.title("Sum of Copies by Product name")
plt.xlabel("Sum of Copies")
plt.ylabel("Product name")
plt.tight_layout()
plt.show()

# -------------------
# 4) Bar chart: Sum of Charged amount by Product name
#    (Skip if Charged amount is empty in CSV)
# -------------------
if df["Charged amount"].notna().sum() == 0:
    print("\nNOTE: 'Charged amount' is completely empty in your CSV, so the Charged Amount chart can't be reproduced.")
    print("Export data again including charged amounts (or provide a file with that column filled).")
else:
    amt_by_product = df.groupby("Product name")["Charged amount"].sum().sort_values()

    plt.figure()
    plt.barh(amt_by_product.index, amt_by_product.values)
    plt.title("Sum of Charged amount by Product name")
    plt.xlabel("Sum of Charged amount")
    plt.ylabel("Product name")
    plt.tight_layout()
    plt.show()
